# MATH50003 Numerical Analysis: Problem 3

This problem sheet explores implementation of triangular solves,
supporting a matrix with two super-diagonals, as well as
permutation and Householder reflections that can be applied to a vector in
$O(n)$ complexity.

In [1]:
using LinearAlgebra, Test, ColorBitstring

# We will override these functions below
import Base: getindex, setindex!, size, *, \

## 1. Dense Matrices

**Problem 1.1** Show that `A*x` is not
implemented as `mul(A, x)` from the lecture notes
by finding a `Float64` example  where the bits do not match.

In [2]:
function mul(A, x)
    m,n = size(A)
    c = zeros(eltype(x), m) # eltype is the type of the elements of a vector/matrix
    for j = 1:n, k = 1:m
        c[k] += A[k, j] * x[j]
    end
    c
end

mul (generic function with 1 method)

In [3]:
n=10000
A = randn(n,n)
x = rand(n)
ans1 = A*x
ans2 = mul(A,x)

10000-element Vector{Float64}:
 -66.21509737451514
   9.83204179220349
  53.82527651919909
 -27.637967072460707
  70.03796093668643
 -43.06781800848224
 -15.702034630524413
 155.7393546818086
  44.494420083877
 -51.316301017445085
  42.794435323241444
 -29.012851554495334
 -86.38901642799807
   ⋮
  -7.754419240590042
   5.642377134042978
  37.24338972829696
  15.632250607672546
  35.71289259782185
 -17.23657704953359
 -57.08740136009069
  42.160844715282195
 -74.4338470233982
  -8.428138399639208
 117.74235181338253
  39.83433219768665

In [4]:
printbits(ans1[1])

1100000001010000100011011100010000100111110001110011111111110010

In [5]:
printbits(ans2[1])

1100000001010000100011011100010000100111110001110011111111011111

## 2. Triangular Matrices

**Problem 2.1** Complete the following functions for solving linear systems with
triangular systems by implementing back and forward-substitution:

In [6]:
function ldiv(U::UpperTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    ## TODO: populate x using back-substitution
    
    for k = n:-1:1
        r = b[k]
        for j = k+1:n
            r -= U[k,j]x[j]
        end
    x[k] = r/U[k,k]
    end
            
    x
end

function ldiv(U::LowerTriangular, b)
    n = size(U,1)
    
    if length(b) != n
        error("The system is not compatible")
    end
        
    x = zeros(n)  # the solution vector
    for k = 1:n
        r = b[k]
        for j = 1:k-1
            r -= U[k,j]*b[j]
        end
    x[k] = r/U[k,k]
    end
    
    x
end

A = [1 2 3;
     4 5 6;
     7 8 9]
U = UpperTriangular(A)
L = LowerTriangular(A)
ldiv(U, [1,2,3])


3-element Vector{Float64}:
 0.0
 0.0
 0.3333333333333333

In [7]:
ldiv(L, [1,2,3])

3-element Vector{Float64}:
  1.0
 -0.4
 -2.2222222222222223

**Problem 2.2⋆** Given $𝐱 ∈ ℝ^n$, find a lower triangular matrix of the form
$$
L = I - 2 𝐯 𝐞_1^⊤
$$
such that:
$$
L 𝐱 = x_1 𝐞_1.
$$
What does $L𝐲$ equal if $𝐲  ∈ ℝ^n$ satisfies $y_1 = 𝐞_1^⊤ 𝐲 = 0$?

## 3. Banded matrices

**Problem 3.1** Complete the implementation of `UpperTridiagonal` which represents a banded matrix with
bandwidths $(l,u) = (0,2)$:

In [8]:
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    # TODO: return U[k,j]
    if k == j
        d[k]
    elseif k + 1 == j
        du[k]
    elseif k + 2 == j
        du2[k]
    else
        0
    end
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end
    
    # TODO: modify d,du,du2 so that U[k,j] == v
    if k == j
        d[k] = v
    elseif k + 1 == j
        du[k] = v
    elseif k + 2 == j
        du2[k] = v
    else
        "Failed"
    end
    U # by convention we return the matrix
end

U = UpperTridiagonal([1,2,3,4,5], [1,2,3,4], [1,2,3])
# setindex!(U,23,2,2)
# getindex(U,2,2)

5×5 UpperTridiagonal{Int64}:
 1  1  1  0  0
 0  2  2  2  0
 0  0  3  3  3
 0  0  0  4  4
 0  0  0  0  5

**Problem 3.2** Complete the following implementations of `*` and `\` for `UpperTridiagonal` so that
they take only $O(n)$ operations.

In [9]:
function *(U::UpperTridiagonal, x::AbstractVector)
    T = promote_type(eltype(U), eltype(x)) # make a type that contains both the element type of U and x
    b = zeros(T, size(U,1)) # returned vector
    d,du,du2 = U.d,U.du,U.du2
    # TODO: populate b so that U*x == b (up to rounding)
    for k = 1:size(d)
        if k == 1 || k == size(d)
            b[k] = x[k]*(d[k])
        elseif k == 2 || k == size(d) - 1
            b[k] = x[k]*(d[k] + du[k-1])
        else
            b[k] = x[k]*(d[k] + du[k-1] + du2[k-1])
        end
    end
    b
end

function \(U::UpperTridiagonal, b::AbstractVector)
    T = promote_type(eltype(U), eltype(b)) # make a type that contains both the element type of U and b
    x = zeros(T, size(U,2)) # returned vector
    # TODO: populate x so that U*x == b (up to rounding)
end

U*[1,2,3,4]

LoadError: MethodError: no method matching (::Colon)(::Int64, ::Tuple{Int64})
[0mClosest candidates are:
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T<:Real at C:\Users\yousi\AppData\Local\Programs\Julia-1.7.1\share\julia\base\range.jl:41
[0m  (::Colon)(::A, ::Any, [91m::C[39m) where {A<:Real, C<:Real} at C:\Users\yousi\AppData\Local\Programs\Julia-1.7.1\share\julia\base\range.jl:10
[0m  (::Colon)(::T, ::Any, [91m::T[39m) where T at C:\Users\yousi\AppData\Local\Programs\Julia-1.7.1\share\julia\base\range.jl:40
[0m  ...

## 4. Permutations

**Problem 4.1⋆** What are the permutation matrices corresponding to the following permutations?
$$
\begin{pmatrix}
1 & 2 & 3 \\
3 & 2 & 1
\end{pmatrix}, \begin{pmatrix}
1 & 2 & 3 & 4 & 5 & 6\\
2 & 1 & 4 & 3 & 6 & 5
\end{pmatrix}.
$$


**Problem 4.2** Complete the implementation of a type representing
permutation matrices that supports `P[k,j]` and such that `*` takes $O(n)$ operations.

In [10]:
struct PermutationMatrix <: AbstractMatrix{Int}
    p::Vector{Int} # represents the permutation whose action is v[p]
    function PermutationMatrix(p::Vector)
        sort(p) == 1:length(p) || error("input is not a valid permutation")
        new(p)
    end
end

size(P::PermutationMatrix) = (length(P.p),length(P.p))

function getindex(P::PermutationMatrix, k::Int, j::Int)
    # TODO: Return P[k,j]
    if j == P.p[k]
        1
    else
        0
    end
end

function *(P::PermutationMatrix, x::AbstractVector)
    # TODO: permute the entries of x
    for i = 1:size(P)[1]
        b[i] = x[P.p[i]]
    end
    b
end

# If your code is correct, this "unit test" will succeed
p = [1, 4, 2, 5, 3]
P = PermutationMatrix(p)
@test P == I(5)[p,:]

Test Passed
  Expression: P == (I(5))[p, :]
   Evaluated: [1 0 … 0 0; 0 0 … 1 0; … ; 0 0 … 0 1; 0 0 … 0 0] == sparse([1, 3, 5, 2, 4], [1, 2, 3, 4, 5], Bool[1, 1, 1, 1, 1], 5, 5)

## 5. Orthogonal matrices

**Problem 5.1⋆** Show that orthogonal matrices preserve the 2-norm of vectors:
$$
\|Q 𝐯\| = \|𝐯\|.
$$


**Problem 5.2⋆** Show that the eigenvalues $λ$ of an orthogonal matrix $Q$ are
on the unit circle: $|λ| = 1$.


**Problem 5.3⋆** Explain why an orthogonal matrix $Q$ must be equal to $I$ if all its eigenvalues are 1.


**Problem 5.4** Complete the implementation of a type representing
reflections that supports `Q[k,j]` and such that `*` takes $O(n)$ operations.

In [11]:
# Represents I - 2v*v'
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T}
end

Reflection(x::Vector{T}) where T = Reflection{T}(x/norm(x))

size(Q::Reflection) = (length(Q.v),length(Q.v))

function getindex(Q::Reflection, k::Int, j::Int)
    if k == j
        1 - 2Q.v[k]*Q.v[j]
    else
        - 2Q.v[k] * Q.v[j]
    end
    # TODO: Return Q[k,j]
end

function *(P::Reflection, x::AbstractVector)
    # TODO: permute the entries of x
    x - 2Q.v * dot(Q.v, x)
end

# If your code is correct, these "unit tests" will succeed
x = randn(5)
Q = Reflection(x)
v = x/norm(x)
@test Q == I-2v*v'
@test Q*v ≈ -v
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [0.9999999999999999 -5.551115123125783e-17 … 2.0816681711721685e-17 2.7755575615628914e-17; -5.551115123125783e-17 0.9999999999999998 … 1.3877787807814457e-16 5.551115123125783e-17; … ; 2.0816681711721685e-17 1.3877787807814457e-16 … 0.9999999999999999 -2.7755575615628914e-17; 2.7755575615628914e-17 5.551115123125783e-17 … -2.7755575615628914e-17 0.9999999999999999] ≈ UniformScaling{Bool}(true)

**Problem 5.5** Complete the following implementation of a Housholder reflection  so that the
unit tests pass.

In [28]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: implement Householder reflection, returning a Reflection
    tmp = zeros(length(x))
    if s
        tmp[1] = -norm(x)
    else
        tmp[1] = norm(x)
    end
    tmp2 = x + tmp
    Reflection(tmp2)
    
end

x = randn(5)
print(norm(x))
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ norm(x)*[1; zeros(length(x)-1)]

Q = householderreflection(false, x)
@test Q isa Reflection
@test Q*x ≈ -norm(x)*[1; zeros(length(x)-1)]

2.2391399988575724

Test Passed
  Expression: Q * x ≈ -(norm(x)) * [1; zeros(length(x) - 1)]
   Evaluated: [-2.2391399988575724, 0.0, -1.3877787807814457e-17, 2.220446049250313e-16, -2.7755575615628914e-17] ≈ [-2.2391399988575724, -0.0, -0.0, -0.0, -0.0]